# Visualization of networks
The visualization is based on the precision matrices

### Imports

In [ ]:
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import scipy as sp

from copy import deepcopy
import copy
import matplotlib.pyplot as plt

import utils

## Load data

In [ ]:
dataset = 'train'
test = False if dataset == 'train' else False

Loading training data

In [ ]:
results_ern_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}_id_clean.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}_id_clean.pkl")
results_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}_id_clean.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}_id_clean.pkl")

results_ern_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_models_{dataset}_id_clean.pkl")
results_ern_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_cov_fal_models_{dataset}_id_clean.pkl")


train_datasets = [results_ern_df, results_ern_lat_demo_df, results_crn_df, results_crn_lat_demo_df, results_ern_crn_df, results_ern_crn_lat_demo_df]
# train_datasets = [results_ern_lat_demo_df, results_crn_lat_demo_df, results_ern_crn_lat_demo_df]

Loading testing data

In [ ]:
results_ern_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_test_id_clean.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test_id_clean.pkl")
results_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_test_id_clean.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test_id_clean.pkl")

results_ern_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_models_test_id_clean.pkl")
results_ern_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_crn_cov_fal_models_test_id_clean.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())
results_ern_crn_lat_demo_test_df['e_LT_F2_C'] = results_ern_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_ern_crn_lat_demo_test_df['e_LT_F2_C'].mean())


test_datasets = [results_ern_test_df, results_ern_lat_demo_test_df, results_crn_test_df, results_crn_lat_demo_test_df, results_ern_crn_test_df, results_ern_crn_lat_demo_test_df]
# test_datasets = [results_ern_lat_demo_test_df, results_crn_lat_demo_test_df, results_ern_crn_lat_demo_test_df]

Loading precision matrixes

In [ ]:
analysis = 'main'
##### MAIN MODELS ###################################
ern_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_0_ntrials_10000_sklearn_cv3_th0.65_without_skewd.pkl')
ern_cov_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_0_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov.pkl')
crn_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_1_ntrials_10000_sklearn_cv3_th0.65_without_skewd.pkl')
crn_cov_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_1_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov.pkl')
ern_control_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_2_ntrials_10000_sklearn_cv3_th0.65_without_skewd.pkl')
ern_cov_control_model_df = pd.read_pickle(f'../nets_structures_results/{analysis}/precision_matrixes/{dataset}/precision_matrix_2_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov.pkl')

precision_matrixes = [
    ern_model_df,
    ern_cov_model_df,
    crn_model_df,
    crn_cov_model_df,
    ern_control_model_df,
    ern_cov_control_model_df
]

##### GENDER MODELS ##############################
# ern_cov_model_df = pd.read_pickle(f'../nets_structures_results/gender/precision_matrixes/{dataset}/precision_matrix_0_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov_sex.pkl')
# crn_cov_model_df = pd.read_pickle(f'../nets_structures_results/gender/precision_matrixes/{dataset}/precision_matrix_1_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov_sex.pkl')
# ern_cov_control_model_df = pd.read_pickle(f'../nets_structures_results/gender/precision_matrixes/{dataset}/precision_matrix_2_ntrials_10000_sklearn_cv3_th0.65_without_skewd_cov_sex.pkl')

# precision_matrixes = [
#     ern_cov_model_df,
#     crn_cov_model_df,
#     ern_cov_control_model_df
# ]

### Visualize precision matrixes

In [ ]:
mapping = {
    "RRQ": "RRQ",
    "DASS-21 Stress": "STR",
    "DASS-21 Dep": "DEP",
    "STAI-T": "ANX",
    "STAI-S Diff": 'AFL',
    "BIS": "BIS",
    "OBSESS": "OBS",
    "HOARD": "HRD",
    "ORD": "ORD",
    "CHECK": "CHK",
    "WBSI": "TSU",
    "IUS-P": "PIU",
    "IUS-I": "IIU",
    "SES": "SES",
    'BAS_D': "DRV",
    'BAS_PRZY': "FUN",
    'BAS_NAG': "RER",
    'INDEC_F': "IND",
    'PUN': "PUN",
    'REW': "REW",
    'HARM': "HRM",
    'T-CTR': "TCR",
    "OT": "THR",
    'OB_PERF': "PER",
    'PS': "STN",
    'G_SE': "GLT",
    'AMB': "AMB",
    'PRED': "NPR",
    'STAND': "STD",   
    "Age": "AGE",
    "Handness": "HND",
    'e_ERN': "ERN",
    'e_LT_F': "LAT",
    'performance': "PRF",
    'e_CRN': "CRN",
    'e_LT_F2_C': "LAT",
    'performance': "PRF",
    'Sex': 'SEX'
}

control_mapping = {
    "RRQ": "RRQ",
    "DASS-21 Stress": "STR",
    "DASS-21 Dep": "DEP",
    "STAI-T": "ANX",
    "STAI-S Diff": 'AFL',
    "BIS": "BIS",
    "OBSESS": "OBS",
    "HOARD": "HRD",
    "ORD": "ORD",
    "CHECK": "CHK",
    "WBSI": "TSU",
    "IUS-P": "PIU",
    "IUS-I": "IIU",
    "SES": "SES",
    'BAS_D': "DRV",
    'BAS_PRZY': "FUN",
    'BAS_NAG': "RER",
    'INDEC_F': "IND",
    'PUN': "PUN",
    'REW': "REW",
    'HARM': "HRM",
    'T-CTR': "TCR",
    "OT": "THR",
    'OB_PERF': "PER",
    'PS': "STN",
    'G_SE': "GLT",
    'AMB': "AMB",
    'PRED': "NPR",
    'STAND': "STD",   
    "Age": "AGE",
    "Handness": "HND",
    'e_ERN': "ERN",
    'e_LT_F': "LT-E",
    'performance': "PRF",
    'e_CRN': "CRN",
    'e_LT_F2_C': "LT-C",
    'performance': "PRF",
    'Sex': 'SEX'
}

ern_order = ['ERN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD']
ern_cov_order = ['ERN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'AGE', 'HND', 'PRF', 'LT-E']
crn_order = ['CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD']
crn_cov_order = ['CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'AGE', 'HND', 'PRF', 'LT-C']
ern_control_order = ['ERN', 'CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD']
ern_cov_control_order = ['ERN', 'CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'AGE', 'HND', 'PRF', 'LT-E', 'LT-C']

orders = [ern_order, ern_cov_order, crn_order, crn_cov_order, ern_control_order, ern_cov_control_order]

ern_cov_gender_order = ['ERN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'SEX', 'AGE', 'HND', 'PRF', 'LT-E']
crn_cov_gender_order = ['CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'SEX', 'AGE', 'HND', 'PRF', 'LT-C']
ern_crn_cov_gender_order = ['ERN', 'CRN', 'RRQ', 'STR', 'DEP', 'ANX', 'AFL', 'BIS', 'OBS', 'HRD', 'ORD', 'CHK', 'TSU', 'PIU', 'IIU', 'SES', 'DRV', 'FUN', 'RER', 'IND', 'PUN', 'REW', 'HRM', 'TCR', 'THR', 'PER', 'STN', 'GLT', 'AMB', 'NPR', 'STD', 'SEX', 'AGE', 'HND', 'PRF', 'LT-E', 'LT-C']


order_gender = [ern_cov_gender_order, crn_cov_gender_order, ern_crn_cov_gender_order]

In [ ]:
sns.set(rc={'figure.figsize':(30 , 30)})

directory = '../new_results/precision_matrixes/gender'
os.makedirs(directory, exist_ok=True)

current_mapping = control_mapping

for i, (matrix, order) in enumerate(zip(precision_matrixes, order_gender)):
    plt.figure(i)
    this_matrix = matrix.rename(columns=current_mapping, index=current_mapping).reindex(columns=order, index=order)
    this_matrix.to_csv(os.path.join(directory, f'precision_matrix_{i}.csv'))
    # sns.heatmap(
    #     data = this_matrix,
    #     annot=True        
    # )

### Visualize partial correlations
Based on the precision matrices

In [ ]:
def get_partial_correlations(precision_matrix):
    """
    Computes partial correlations based on the inverse covariance estimates.
    
    Parameters
    ----------
    precision_matrix : pd.DataFrame
        A square DataFrame that represents inverse covariance matrix. Columns and indexes are node names.
    
    Returns
    -------
    partial_correlation_matrix : pd.DataFrame
        A square DataFrame that represents partial correlations matrix. Columns and indexes are node names.

    """
    partial_correlation_matrix = precision_matrix.copy()
    d = 1 / np.sqrt(np.diag(partial_correlation_matrix))
    partial_correlation_matrix *= d
    partial_correlation_matrix *= d[:, np.newaxis]
    partial_correlation_matrix = -partial_correlation_matrix

    return partial_correlation_matrix

In [ ]:
partial_correlation_matrices = [get_partial_correlations(precision_matrix) for precision_matrix in precision_matrixes]

In [ ]:
sns.set(rc={'figure.figsize':(30 , 30)})

directory = '../new_results/partial_correlation_matrixes'
os.makedirs(directory, exist_ok=True)

current_mapping = control_mapping

for i, (matrix, order) in enumerate(zip(partial_correlation_matrices, order_gender)):
    plt.figure(i)
    this_matrix = matrix.rename(columns=current_mapping, index=current_mapping).reindex(columns=order, index=order)
    this_matrix.to_csv(os.path.join(directory, f'partial_correlation_matrix_{i}.csv'))
    
    mask = np.triu(this_matrix)
    # sns.heatmap(
    #     data = this_matrix,
    #     annot=True,
    #     center = 0.0,
    #     cmap='vlag',
    #     mask=mask,
    # )

## Visualize ERN and CRN graphs

### 1. Extract list of links from precision matrices

In [ ]:
if test:
    test_links = []

    for matrix in precision_matrixes_test:
        this_links = utils.get_links(matrix)
        test_links.append(this_links)
else:
    links = []

    for matrix in precision_matrixes:
        this_links = utils.get_links(matrix)
        links.append(this_links)

### 2. Calculate within- and out-of-sample nodes predictability

In [ ]:
within_sample_predictability = [
    utils.calculate_nodes_predictability(X, precision_matrix) 
    for X, precision_matrix in zip(train_datasets, precision_matrixes)
]

out_of_sample_predictability = [
    utils.calculate_nodes_predictability(X, precision_matrix) 
    for X, precision_matrix in zip(test_datasets, precision_matrixes)
]

### 3. Bulid the graph from the links

Node labels mappers for plotting

In [ ]:
control_mapping = {
    "RRQ": "RRQ",
    "DASS-21 Stress": "STR",
    "DASS-21 Dep": "DEP",
    "STAI-T": "ANX",
    "STAI-S Diff": 'AFL',
    "BIS": "BIS",
    "OBSESS": "OBS",
    "HOARD": "HRD",
    "ORD": "ORD",
    "CHECK": "CHK",
    "WBSI": "TSU",
    "IUS-P": "PIU",
    "IUS-I": "IIU",
    "SES": "SES",
    'BAS_D': "DRV",
    'BAS_PRZY': "FUN",
    'BAS_NAG': "RER",
    'INDEC_F': "IND",
    'PUN': "PUN",
    'REW': "REW",
    'HARM': "HRM",
    'T-CTR': "TCR",
    "OT": "THR",
    'OB_PERF': "PER",
    'PS': "STN",
    'G_SE': "GLT",
    'AMB': "AMB",
    'PRED': "NPR",
    'STAND': "STD",   
    "Age": "AGE",
    "Handness": "HND",
    'e_ERN': "ERN",
    'e_LT_F': "LT-E",
    'performance': "PRF",
    'e_CRN': "CRN",
    'e_LT_F2_C': "LT-C",
    'performance': "PRF",
    'Sex': 'SEX'
}

Set saving parameters

In [ ]:
dir_graph_viz = '../new_results/graph_viz_gender'
dir_graphs = '../new_results/graphs/gender'
os.makedirs(dir_graph_viz, exist_ok=True)
os.makedirs(dir_graphs, exist_ok=True)

save_graphs = True

Set plotting parameters

In [ ]:
palette = sns.color_palette("colorblind")

color_mapping = {
    'ERN': palette[1],
    'CRN': palette[1],
    'SEX': palette[2],
    'LT-E': palette[2],
    'LT-C': palette[2],
    'LAT': palette[2],
    'AGE': palette[2],
    'HND': palette[2],
    'PRF': palette[2],
}

In [ ]:
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection

def draw_graph(
    links, 
    basic_links=None, 
    basic=False, 
    mapping=None, 
    seed=1, 
    nodes_predictabilities=None,
    node_colors = None,
    scale=1, 
    specific_positions = [], 
    layout = None,
    nx_layout=None,
    save=None,
    dir=None,
):
    """
    Draws graph based on the provided links DataFrame

    Parameters
    ----------
    links : pd.DataFrame
        A DataFrame that consists of 3 columns: var1, var2, and weight
    basic_links : pd.DataFrame, optional
        A link DataFrame for links to be drawn in a special way. Default is None.
    basic : bool or str, optional
        Indicates how to draw basic_links. 
        'first' draws basic_links bolded to make them more distinct.
        'only' draws only basic_links. Default is False.
    mapping : dict, optional
        Dictionary where the keys are nodes names from links DataFrame, and values are the node names to be drawn on the graph.
        Default is None.
    seed : int, optional
        Seed to be passed to nx.spring_layout. Default is 1.
    nodes_predictabilities: pd.DataFrame, optional
        Predictabilities to be drawn around the nodes. The DataFrame has nodes as columns and predictabilities ac values. Default is None.
    scale : float, optional
        Scale to be passed to nx.spring_layout. Default is 1.
    specific_positions : List[Tuple[str, List[float]]], optional
        Set positions of provided nodes. List of tuples in a form: [('node_name', [x, y])]. Default is [].
    layout : List[Tuple[str, List[float]]], optional
        Layout of the graph. Must contain positions of all nodes in the graph. If provided, it replaces the default nx.spring_layout.
        Default is None.
    save : str or None, optional
        Whether to save graph figure. If a string is provided, it is assumed to be a path where to save the figure. Default is None.

    Returns
    -------
    pos : List[Tuple[str, List[float]]]
        Layout of the graph.
    G: nx.Graph
        Graph created based on the provided links
    """
    
    # set plotting parameters
    cm = 1/2.54
    dpi = 700
    plt.rcParams['figure.dpi'] = dpi
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams['ytick.labelsize'] = 7
    plt.rcParams['xtick.labelsize'] = 7
    plt.rcParams['axes.labelsize'] = 9
    plt.rcParams["axes.edgecolor"] = ".15"
    plt.rcParams["axes.linewidth"]  = 0.5
    sns.set_style("white")
    palette = sns.color_palette("colorblind")
    
          
    fig = plt.figure(3, figsize=(10*cm, 10*cm))
    ax = fig.add_axes([0,0,1,1], aspect=1)
    ax.set_xlim([-1,1])
    ax.set_ylim([-1,1])
    
    # create graph
    G = nx.from_pandas_edgelist(links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
    G = nx.relabel_nodes(G, mapping) if mapping is not None else G
    
    # set nodes colors
    node_color = [node_colors[node] if node in node_colors else palette[0] for node in G.nodes()]
    # set nodes position
    edges = G.edges() 
    
    for u,v in edges:
        G[u][v]['weight'] = abs(G[u][v]['weight'])
    
    if (layout == None) & (nx_layout == None):
        pos_i = nx.spring_layout(G, k=60*(1/np.sqrt(len(G.nodes()))), iterations=1000, scale=scale, seed=seed)
        pos = pos_i
        # set specific positions, if desired
        for position in specific_positions:
            pos[position[0]] = position[1]
    elif (layout != None) & (nx_layout == None):
        pos = layout
    else:
        pos = nx_layout(G, scale=scale)

    # if basic is False, draw all links
    if basic is False:
        G = nx.from_pandas_edgelist(links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
        G = nx.relabel_nodes(G, mapping) if mapping is not None else G
        
        edges = G.edges() 
           
        weights = []
        for u,v in edges:
            weight = G[u][v]['weight']
            # print(f"{u},{v},{weight}")
            weights.append(weight)
            
        edge_color = [('red' if edge < 0 else 'blue') for edge in weights]

        weights_alphas = []
        weight_bold = []
        
        # set the width of links based on the weights
        for edge_weight in weights:
            if abs(edge_weight) < 0.05:
                alpha = 0.2 
                bold = 1
            elif abs(edge_weight) >= 0.05 and abs(edge_weight) < 0.1:
                alpha = 0.25
                bold = 1
            elif abs(edge_weight) >= 0.1 and abs(edge_weight) < 0.15:
                alpha = 0.3
                bold = 2
            elif abs(edge_weight) >= 0.15 and abs(edge_weight) < 0.2:
                alpha = 0.35
                bold = 1
            elif abs(edge_weight) >= 0.15 and abs(edge_weight) < 0.2:
                alpha = 0.38
                bold = 1
            elif abs(edge_weight) >= 0.2 and abs(edge_weight) < 0.25:
                alpha = 0.40
                bold = 1
            elif abs(edge_weight) >= 0.25 and abs(edge_weight) < 0.3:
                alpha = 0.42
                bold = 1
            elif abs(edge_weight) >= 0.3 and abs(edge_weight) < 0.35:
                alpha = 0.45
                bold = 1
            else:
                alpha = 0.5
                bold = 1
            weights_alphas.append(alpha)
            weight_bold.append(bold)

    # if basic is not False, draw links provided as 'basic_links'  using one of the following startegies:'first' | 'only'
    if (basic is not False):
        G = nx.from_pandas_edgelist(links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
        G = nx.relabel_nodes(G, mapping) if mapping is not None else G
        
        edges = G.edges() 
           
        weights = []
        for u,v in edges:
            weight = G[u][v]['weight']
            weights.append(weight)
            
        edge_color = [('red' if edge < 0 else 'blue') for edge in weights]
        
        weights_alphas = []
        weight_bold = []

        for edge_weight in weights:
            alpha = 0.15 
            bold = 1
            weights_alphas.append(alpha)
            weight_bold.append(bold)
            
        ###########################################################################################################
        G_eeg = nx.from_pandas_edgelist(basic_links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
        G_eeg = nx.relabel_nodes(G_eeg, mapping) if mapping is not None else G_eeg
        
        eeg_edges = G_eeg.edges() 
        
        weights_eeg = []
        for u,v in eeg_edges:
            weight = G_eeg[u][v]['weight']
            print(f"{u},{v},{weight}")
            weights_eeg.append(weight)
            
        edge_eeg_color = [('red' if edge < 0 else 'blue') for edge in weights_eeg]
        
        weights_eeg_alphas = []
        weights_eeg_bold = []

        for edge_weight in weights_eeg:
            if abs(edge_weight) < 0.05:
                alpha = 0.3 
                bold = 2
            elif abs(edge_weight) >= 0.05 and abs(edge_weight) < 0.09:
                alpha = 0.35
                bold = 2
            elif abs(edge_weight) >= 0.09 and abs(edge_weight) < 0.16:
                alpha = 0.40
                bold = 3
            elif abs(edge_weight) >= 0.16 and abs(edge_weight) < 0.5:
                alpha = 0.45
                bold = 3
            else:
                alpha = 0.6
                bold = 3
            weights_eeg_alphas.append(alpha)
            weights_eeg_bold.append(bold)
            
        
    nx.draw_networkx_nodes(
        G, 
        pos=pos,
        linewidths=0.5,
        edgecolors='black',
        node_size = 210,
        node_color=node_color,
    )

    nx.draw_networkx_labels(
        G, 
        pos=pos,
        font_size=6,
    )
    
    if basic == False:
        nx.draw_networkx_edges(
            G, 
            pos=pos,
            edgelist = G.edges(),
            edge_color = edge_color,
            alpha=weights_alphas,
            width = weight_bold,
        )
        

    if basic_links is not None and basic == 'only':
        nx.draw_networkx_edges(
            G, 
            pos=pos,
            edgelist = G_eeg.edges(),
            edge_color = edge_eeg_color,
            alpha=weights_eeg_alphas,
            width = weights_eeg_bold,
        )
    
    if basic_links is not None and basic == 'first':
        nx.draw_networkx_edges(
            G, 
            pos=pos,
            edgelist = G.edges(),
            edge_color = edge_color,
            alpha=weights_alphas,
            width = weight_bold,
        )
        
        nx.draw_networkx_edges(
            G, 
            pos=pos,
            edgelist = G_eeg.edges(),
            edge_color = edge_eeg_color,
            alpha=weights_eeg_alphas,
            width = weights_eeg_bold,
        )
    
    # add nodes predictabilities
    if nodes_predictabilities is not None:
        nodes_predictabilities = nodes_predictabilities.rename(columns=mapping) if mapping is not None else nodes_predictabilities

        patches = []
        colors = []
        for item in pos.items():
            node_id = item[0]
            node_predictability = nodes_predictabilities[node_id].to_list()[0]
            node_predictability_percent = 360 * node_predictability
            cor_x, cor_y = item[1]

            weg_bck = Wedge((cor_x, cor_y), .065, 0, 360, width=0.03, edgecolor='black', linewidth=0.3, facecolor='white')
            patches.append(weg_bck)
            colors.append(0)
            ax.add_patch(weg_bck)

            weg = Wedge((cor_x, cor_y), .065, 0, node_predictability_percent, width=0.03, edgecolor='black', linewidth=0.3, facecolor='gray')
            patches.append(weg)
            colors.append(1)
            ax.add_patch(weg)

    plt.show()

    if save != None:
        fig.savefig(os.path.join(dir, f'{save}.png'), bbox_inches='tight')

    return pos, G

#### ERN + CRN cov graph

In [ ]:
layout = {
    'ERN': np.array([0.72154533, 0.1473676 ]),
    'CRN': np.array([0.58204701, 0.57105916]),
    'LT-E': np.array([0.81985766, 0.42047284]),
    'RRQ': np.array([0.48874008, 0.19843229]),
    'OBS': np.array([ 0.42739243, -0.07231427]),
    'REW': np.array([-0.66788538,  0.26973331]),
    'LT-C': np.array([0.08266476, 0.6662342 ]),
    'DRV': np.array([-0.36565729,  0.44197988]),
    'ORD': np.array([-0.4203207 , -0.52003789]),
    'BIS': np.array([0.09287787, 0.15483977]),
    'FUN': np.array([-0.4008687 ,  0.07773721]),
    'RER': np.array([-0.4341017 ,  0.28789437]),
    'CHK': np.array([-0.09881527, -0.72674955]),
    'PER': np.array([-0.06245429, -0.37225267]),
    'AGE': np.array([-0.70101968,  0.66191661]),
    'HRD': np.array([ 0.14667322, -0.43049883]),
    'IND': np.array([0.09504048, 0.39430738]),
    'STN': np.array([-0.48860978, -0.06525376]),
    'HND': [-0.9, -0.1],
    'AFL': np.array([ 0.66387188, -0.67282584]),
    'THR': np.array([ 0.08588435, -0.01106539]),
    'TSU': np.array([ 0.44708268, -0.21070759]),
    'AMB': np.array([-0.33349838, -0.13061421]),
    'STR': np.array([ 0.21233232, -0.12415112]),
    'DEP': np.array([0.2634654 , 0.04249963]),
    'ANX': np.array([0.20643209, 0.2985926 ]),
    'SES': np.array([-0.05782301,  0.19769243]),
    'PIU': np.array([-0.19181228, -0.17929032]),
    'PUN': np.array([0.18641841, 0.57131294]),
    'IIU': np.array([-0.14720288,  0.33094186]),
    'GLT': np.array([-0.26757993, -0.44751226]),
    'TCR': np.array([ 0.27314848, -0.48694078]),
    'STD': np.array([-0.63220449, -0.31930234]),
    'NPR': np.array([-0.4852652 , -0.28456157]),
    'PRF': [0.9, 0.7],
    'HRM': [0.1, -0.7],
    'SEX': np.array([0.58204701, 0.8])
}

In [ ]:
positions = [
    ('HRM', [.1, -0.7]),
    ('HND', [-0.9, -0.1]),
    ('PRF', [0.9, 0.7]),
]


current_links = links[2]
current_out_of_sample_predictability = out_of_sample_predictability[2]
current_within_sample_predictability = within_sample_predictability[2]

ern_crn_cov_layout, ern_crn_cov_G = draw_graph(
    current_links, 
    seed=13, # 2,11, 16
    scale=1,
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0),
    basic='first', 
    nodes_predictabilities = current_out_of_sample_predictability, 
    specific_positions=positions,
    mapping=control_mapping,
    node_colors=color_mapping,
    save = 'ern_crn_cov_gender' if save_graphs else None,
    dir=dir_graph_viz,
    layout=layout
    # nx_layout = nx.circular_layout,
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(ern_crn_cov_G, os.path.join(dir_graphs, 'ern_crn_cov_gender_graph.gml'))

#### ERN + CRN graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout = ern_crn_cov_layout.copy()

# remove covariates nodes from the layout
layout.pop('AGE', None)
layout.pop('HND', None)
layout.pop('PRF', None)
layout.pop('LT-E', None)
layout.pop('LT-C', None)

In [ ]:
current_links = links[4]
current_out_of_sample_predictability = out_of_sample_predictability[4]
current_within_sample_predictability = within_sample_predictability[4]

ern_crn_layout, ern_crn_G = draw_graph(
    current_links, 
    seed=13, # 2,11, 16
    scale=1,
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0),
    basic='first', 
    nodes_predictabilities = current_out_of_sample_predictability, 
    specific_positions=positions,
    mapping=control_mapping,
    node_colors=color_mapping,
    save = 'ern_crn' if save_graphs else None,
    dir=dir_graph_viz,
    layout = layout
    # nx_layout = nx.circular_layout,
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(ern_crn_G, os.path.join(dir_graphs, 'ern_crn_graph.gml'))

#### ERN graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout = ern_crn_cov_layout.copy()

# remove covariates nodes from the layout
layout.pop('AGE', None)
layout.pop('HND', None)
layout.pop('PRF', None)
layout.pop('LT-E', None)
layout.pop('LT-C', None)
layout.pop('CRN', None)

In [ ]:
current_links = links[0]
current_out_of_sample_predictability = out_of_sample_predictability[0]
current_within_sample_predictability = within_sample_predictability[0]

ern_layout, ern_G = draw_graph(
    current_links, 
    scale=1.1,
    seed=13, 
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0),
    basic='first', 
    nodes_predictabilities = current_out_of_sample_predictability, 
    mapping=mapping,     
    layout = layout,
    save='ern' if save_graphs else None,
    node_colors=color_mapping,
    dir=dir_graph_viz
) 

Display the predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(ern_G, os.path.join(dir_graphs, 'ern_graph.gml'))

#### CRN graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout = ern_crn_cov_layout.copy()

# remove covariates nodes from the layout
layout.pop('AGE', None)
layout.pop('HND', None)
layout.pop('PRF', None)
layout.pop('LT-E', None)
layout.pop('LT-C', None)
layout.pop('ERN', None)

In [ ]:
current_links = links[2]
current_out_of_sample_predictability = out_of_sample_predictability[2]
current_within_sample_predictability = within_sample_predictability[2]

_, crn_G = utils.draw_graph(
    current_links, 
    seed=16, 
    scale=1.1,
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0), 
    basic='first', 
    mapping=mapping, 
    nodes_predictabilities = current_out_of_sample_predictability,
    specific_positions=positions,
    layout=layout,
    save = 'crn' if save_graphs else None,
    node_colors=color_mapping,
    dir=dir_graph_viz
) 

Display predictability

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(crn_G, os.path.join(dir_graphs, 'crn_graph.gml'))

#### CRN cov graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout_crn = layout.copy()
layout_crn.pop('LT-E', None)
layout_crn.pop('ERN', None)

In [ ]:
current_links = links[1]
current_out_of_sample_predictability = out_of_sample_predictability[1]
current_within_sample_predictability = within_sample_predictability[1]


crn_cov_layout, crn_cov_G = utils.draw_graph(
    current_links, 
    seed=8, 
    scale=1.1,
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0), 
    basic='first', 
    mapping=control_mapping,
    nodes_predictabilities = current_out_of_sample_predictability,
    specific_positions = positions,
    layout = layout_crn,
    save = 'crn_cov_gender' if save_graphs else None,
    node_colors=color_mapping,
    dir=dir_graph_viz
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(crn_cov_G, os.path.join(dir_graphs, 'crn_cov_gender_graph.gml'))

#### ERN cov graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout_ern = layout.copy()
layout_ern.pop('LT-C', None)
layout_ern.pop('CRN', None)

In [ ]:
current_links = links[0]
current_out_of_sample_predictability = out_of_sample_predictability[0]
current_within_sample_predictability = within_sample_predictability[0]


_, ern_cov_G = utils.draw_graph(
    current_links, 
    seed=8, 
    scale=1.1,
    basic_links=utils.leave_n_edges_from_eeg(current_links, N=0), 
    basic='first', 
    mapping=control_mapping,
    nodes_predictabilities = current_out_of_sample_predictability,
    specific_positions = positions,
    layout = layout_ern,
    save = 'ern_cov_gender|' if save_graphs else None,
    node_colors=color_mapping,
    dir=dir_graph_viz
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_within_sample_predictability)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(current_out_of_sample_predictability)

Save graph

In [ ]:
nx.write_gml(ern_cov_G, os.path.join(dir_graphs, 'ern_cov_gender_graph.gml'))

---

## Visualize ERN and CRN graphs difference

Define node labels mappers

In [ ]:
mapping_difference_graphs = {
    "RRQ": "RRQ",
    "DASS-21 Stress": "STR",
    "DASS-21 Dep": "DEP",
    "STAI-T": "ANX",
    "STAI-S Diff": 'AFL',
    "BIS": "BIS",
    "OBSESS": "OBS",
    "HOARD": "HRD",
    "ORD": "ORD",
    "CHECK": "CHK",
    "WBSI": "TSU",
    "IUS-P": "PIU",
    "IUS-I": "IIU",
    "SES": "SES",
    'BAS_D': "DRV",
    'BAS_PRZY': "FUN",
    'BAS_NAG': "RER",
    'INDEC_F': "IND",
    'PUN': "PUN",
    'REW': "REW",
    'HARM': "HRM",
    'T-CTR': "TCR",
    "OT": "THR",
    'OB_PERF': "PER",
    'PS': "STN",
    'G_SE': "GLT",
    'AMB': "AMB",
    'PRED': "NPR",
    'STAND': "STD",   
    "Age": "AGE",
    "Handness": "HND",
    'e_ERN': "ERN",
    'e_LT_F': "LAT",
    'performance': "PRF",
    'e_CRN': "ERN",
    'e_LT_F2_C': "LAT",
    'performance': "PRF",
    'Sex': 'SEX'
}

palette = sns.color_palette("colorblind")

color_mapping = {
    'ERN': palette[1],
    'SEX': palette[2],
    'LAT': palette[2],
    'AGE': palette[2],
    'HND': palette[2],
    'PRF': palette[2],
}

### Create graphs from precision matrices

Here, the threshold is set very low so as not to introduce an artificial qualitative difference between ERN and CRN networks when the link is slightly below the threshold in one network (and will be zeroed) and slightly above in the second (and will not be zeroed).

In [ ]:
def create_graph(links, mapping):
    G = nx.from_pandas_edgelist(links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
    G = nx.relabel_nodes(G, mapping)
    return G

In [ ]:
links_for_diffs = [utils.leave_n_edges_from_eeg(utils.get_links(matrix_, threshold=0.005), N=2) for matrix_ in precision_matrixes]
graphs_for_diffs = [create_graph(links, mapping_difference_graphs) for links in links_for_diffs]

### Estimate difference graphs

#### 1. ERN minus CRN model

In [ ]:
save_graphs=True

In [ ]:
difference_graph_ern, colors, styles = utils.estimate_difference_graph(
    graphs_for_diffs[0], 
    graphs_for_diffs[2],
    threshold=0.02
)

utils.draw_difference_graph(
    difference_graph_ern, 
    colors=colors, 
    linestyle=styles, 
    seed=16, 
    no_purple=True, 
    scale=1.2,
    layout = ern_layout,
    node_colors=color_mapping,
    save = 'ern_crn_diff_layout' if save_graphs else None,
    dir=dir_graph_viz
)

Calculate nodes predictability

In [ ]:
G_ = difference_graph_ern.copy()       
ern_crn_diff_matrix = nx.to_numpy_array(G_)
ern_crn_diff_matrix_df = pd.DataFrame(ern_crn_diff_matrix, columns=G_.nodes(), index=G_.nodes())

In [ ]:
ern_crn_diff_within_sample_predictability = utils.calculate_nodes_predictability(train_datasets[0].rename(columns=mapping_difference_graphs), ern_crn_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_diff_within_sample_predictability)

ern_crn_diff_out_of_sample_predictability = utils.calculate_nodes_predictability(test_datasets[0].rename(columns=mapping_difference_graphs), ern_crn_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_diff_out_of_sample_predictability)

Save the graph

In [ ]:
nx.write_gml(difference_graph_ern, os.path.join(dir_graphs, 'ern_crn_diff_graph_layout.gml'))
# ern_crn_diff_matrix_df.to_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_ern_crn_diff_layout.pkl')

#### ERN with covariates minus CRN with covariates

In [ ]:
difference_graph_ern_cov, colors, styles = utils.estimate_difference_graph(
    graphs_for_diffs[0], 
    graphs_for_diffs[1],
    threshold=0.02
)

utils.draw_difference_graph(
    difference_graph_ern_cov, 
    colors=colors, 
    linestyle=styles, 
    seed=7, 
    no_purple=True,
    scale=1.3,
    layout = ern_cov_layout,
    save = 'ern_crn_gender_diff' if save_graphs else None,
    node_colors=color_mapping,
    dir=dir_graph_viz
)

Calculate nodes predictability

In [ ]:
G_ = difference_graph_ern_cov.copy()       
ern_crn_cov_diff_matrix = nx.to_numpy_array(G_)
ern_crn_cov_diff_matrix_df = pd.DataFrame(ern_crn_cov_diff_matrix, columns=G_.nodes(), index=G_.nodes())

In [ ]:
ern_crn_cov_diff_within_sample_predictability = utils.calculate_nodes_predictability(train_datasets[1].rename(columns=mapping_difference_graphs), ern_crn_cov_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_cov_diff_within_sample_predictability)

ern_crn_cov_diff_out_of_sample_predictability = utils.calculate_nodes_predictability(test_datasets[1].rename(columns=mapping_difference_graphs), ern_crn_cov_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_cov_diff_out_of_sample_predictability)

Save the graph

In [ ]:
nx.write_gml(difference_graph_ern_cov, '../data/graphs/ern_crn_cov_diff_graph_layout.gml')
# ern_crn_cov_diff_matrix_df.to_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_ern_crn_cov_diff_layout.pkl')

---
## Draw legend

In [ ]:
color_mapping = {
    'ERN': palette[1],
    'CRN': palette[1],
    'SEX': palette[2],
    'LAT': palette[2],
    'LAT-E': palette[2],
    'LAT-C': palette[2],
    'AGE': palette[2],
    'HND': palette[2],
    'PRF': palette[2],
}

- ERN

In [ ]:
legend_labels = {
    "RRQ: Rumination": "RRQ",
    "STR: Stress": "STR",
    "DEP: Depression": "DEP",
    "ANX: Anxiety trait": "ANX",
    "AFL: Affective load": 'AFL',
    "BIS: Behavioral inhibition": "BIS",
    "OBS: Obsessing": "OBS",
    "HRD: Hoarding": "HRD",
    "ORD: Ordering": "ORD",
    "CHK: Checking": "CHK",
    "TSU: Thought supression": "TSU",
    "PIU: Prospective intolerance of uncertainty": "PIU",
    "IIU: Inhibitory intolerance of uncertainty": "IIU",
    "SES: Self-esteem": "SES",
    'DRV: Drive': "DRV",
    'FUN: Fun-seeking': "FUN",
    'RER: Reward responsiveness': "RER",
    'IND: Indecisivness': "IND",
    'PUN: Punishment sensitivity': "PUN",
    'REW: Reward sensitivity': "REW",
    'HRM: Inflated harm responsibility': "HRM",
    'TCR: Importance of thought control': "TCR",
    "THR: Threat overestimation": "THR",
    'PER: Perfectionism/intolerance of uncertainty': "PER",
    'STN: Personal standards': "STN",
    'GLT: Guilt sensitivity': "GLT",
    'AMB: Avoidance of ambiguity': "AMB",
    'NPR: Need for predictability': "NPR",
    'STD: High standards': "STD",   
    "AGE: Age": "AGE",
    "HND: Handedness": "HND",
    'PRF: Performance': "PRF",
    'LAT: ERN latency': "LAT",
    'ERN: ERN amplitude': "ERN",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

node_color = [color_mapping[node] if node in color_mapping else palette[0] for node in legend_labels.values()]
    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=node_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/ern_legend_NEW.png')

- CRN

In [ ]:
legend_labels = {
    "RRQ: Rumination": "RRQ",
    "STR: Stress": "STR",
    "DEP: Depression": "DEP",
    "ANX: Anxiety trait": "ANX",
    "AFL: Affective load": 'AFL',
    "BIS: Behavioral inhibition": "BIS",
    "OBS: Obsessing": "OBS",
    "HRD: Hoarding": "HRD",
    "ORD: Ordering": "ORD",
    "CHK: Checking": "CHK",
    "TSU: Thought supression": "TSU",
    "PIU: Prospective intolerance of uncertainty": "PIU",
    "IIU: Inhibitory intolerance of uncertainty": "IIU",
    "SES: Self-esteem": "SES",
    'DRV: Drive': "DRV",
    'FUN: Fun-seeking': "FUN",
    'RER: Reward responsiveness': "RER",
    'IND: Indecisivness': "IND",
    'PUN: Punishment sensitivity': "PUN",
    'REW: Reward sensitivity': "REW",
    'HRM: Inflated harm responsibility': "HRM",
    'TCR: Importance of thought control': "TCR",
    "THR: Threat overestimation": "THR",
    'PER: Perfectionism/intolerance of uncertainty': "PER",
    'STN: Personal standards': "STN",
    'GLT: Guilt sensitivity': "GLT",
    'AMB: Avoidance of ambiguity': "AMB",
    'NPR: Need for predictability': "NPR",
    'STD: High standards': "STD",   
    "AGE: Age": "AGE",
    "HND: Handedness": "HND",
    'PRF: Performance': "PRF",
    'LAT: CRN latency': "LAT",
    'CRN: CRN amplitude': "CRN",
    # '2E: CRN': "2E",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color = [color_mapping[node] if node in color_mapping else palette[0] for node in legend_labels.values()]

    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/crn_legend_NEW.png')

- Diff

In [ ]:
legend_labels = {
    "RRQ: Rumination": "RRQ",
    "STR: Stress": "STR",
    "DEP: Depression": "DEP",
    "ANX: Anxiety trait": "ANX",
    "AFL: Affective load": 'AFL',
    "BIS: Behavioral inhibition": "BIS",
    "OBS: Obsessing": "OBS",
    "HRD: Hoarding": "HRD",
    "ORD: Ordering": "ORD",
    "CHK: Checking": "CHK",
    "TSU: Thought supression": "TSU",
    "PIU: Prospective intolerance of uncertainty": "PIU",
    "IIU: Inhibitory intolerance of uncertainty": "IIU",
    "SES: Self-esteem": "SES",
    'DRV: Drive': "DRV",
    'FUN: Fun-seeking': "FUN",
    'RER: Reward responsiveness': "RER",
    'IND: Indecisivness': "IND",
    'PUN: Punishment sensitivity': "PUN",
    'REW: Reward sensitivity': "REW",
    'HRM: Inflated harm responsibility': "HRM",
    'TCR: Importance of thought control': "TCR",
    "THR: Threat overestimation": "THR",
    'PER: Perfectionism/intolerance of uncertainty': "PER",
    'STN: Personal standards': "STN",
    'GLT: Guilt sensitivity': "GLT",
    'AMB: Avoidance of ambiguity': "AMB",
    'NPR: Need for predictability': "NPR",
    'STD: High standards': "STD",   
    "AGE: Age": "AGE",
    "HND: Handedness": "HND",
    'PRF: Performance': "PRF",
    'LAT: ERP latency': "LAT",
    'ERN: ERN amplitude': "ERN",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color = [color_mapping[node] if node in color_mapping else palette[0] for node in legend_labels.values()]

    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/diff_legend_NEW.png')

- Control

In [ ]:
legend_labels = {
    "RRQ: Rumination": "RRQ",
    "STR: Stress": "STR",
    "DEP: Depression": "DEP",
    "ANX: Anxiety trait": "ANX",
    "AFL: Affective load": 'AFL',
    "BIS: Behavioral inhibition": "BIS",
    "OBS: Obsessing": "OBS",
    "HRD: Hoarding": "HRD",
    "ORD: Ordering": "ORD",
    "CHK: Checking": "CHK",
    "TSU: Thought supression": "TSU",
    "PIU: Prospective intolerance of uncertainty": "PIU",
    "IIU: Inhibitory intolerance of uncertainty": "IIU",
    "SES: Self-esteem": "SES",
    'DRV: Drive': "DRV",
    'FUN: Fun-seeking': "FUN",
    'RER: Reward responsiveness': "RER",
    'IND: Indecisivness': "IND",
    'PUN: Punishment sensitivity': "PUN",
    'REW: Reward sensitivity': "REW",
    'HRM: Inflated harm responsibility': "HRM",
    'TCR: Importance of thought control': "TCR",
    "THR: Threat overestimation": "THR",
    'PER: Perfectionism/intolerance of uncertainty': "PER",
    'STN: Personal standards': "STN",
    'GLT: Guilt sensitivity': "GLT",
    'AMB: Avoidance of ambiguity': "AMB",
    'NPR: Need for predictability': "NPR",
    'STD: High standards': "STD",   
    "AGE: Age": "AGE",
    "HND: Handedness": "HND",
    'PRF: Performance': "PRF",
    'LAT-E: ERN latency': "LAT-E",
    'LAT-C: CRN latency': "LAT-C",
    'ERN: ERN amplitude': "ERN",
    'CRN: CRN amplitude': "CRN",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color = [color_mapping[node] if node in color_mapping else palette[0] for node in legend_labels.values()]

    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/control_legend_NEW.png')

- Gender

In [ ]:
legend_labels = {
    "RRQ: Rumination": "RRQ",
    "STR: Stress": "STR",
    "DEP: Depression": "DEP",
    "ANX: Anxiety trait": "ANX",
    "AFL: Affective load": 'AFL',
    "BIS: Behavioral inhibition": "BIS",
    "OBS: Obsessing": "OBS",
    "HRD: Hoarding": "HRD",
    "ORD: Ordering": "ORD",
    "CHK: Checking": "CHK",
    "TSU: Thought supression": "TSU",
    "PIU: Prospective intolerance of uncertainty": "PIU",
    "IIU: Inhibitory intolerance of uncertainty": "IIU",
    "SES: Self-esteem": "SES",
    'DRV: Drive': "DRV",
    'FUN: Fun-seeking': "FUN",
    'RER: Reward responsiveness': "RER",
    'IND: Indecisivness': "IND",
    'PUN: Punishment sensitivity': "PUN",
    'REW: Reward sensitivity': "REW",
    'HRM: Inflated harm responsibility': "HRM",
    'TCR: Importance of thought control': "TCR",
    "THR: Threat overestimation": "THR",
    'PER: Perfectionism/intolerance of uncertainty': "PER",
    'STN: Personal standards': "STN",
    'GLT: Guilt sensitivity': "GLT",
    'AMB: Avoidance of ambiguity': "AMB",
    'NPR: Need for predictability': "NPR",
    'STD: High standards': "STD",
    # 'SEX: Sex': 'SSEX',
    "AGE: Age": "AGE",
    "HND: Handedness": "HND",
    'PRF: Performance': "PRF",
    'LAT: ERP latency': "LAT",
    'ERN: ERN amplitude': "ERN",
    'CRN: CRN amplitude': "CRN",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color = [color_mapping[node] if node in color_mapping else palette[0] for node in legend_labels.values()]

    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/gender_legend_NEW.png')